In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from device import Device
from loralong import LoRaLong

# Set device consumption characteristics
# https://www.semtech.com/products/wireless-rf/lora-connect/sx1262
sx1262 = Device()
sx1262.SetTxCurrent(118.0)          # 868~915 MHz/+22dBm
sx1262.SetRxCurrent(5.3)            # DCDC Mode/Rx Boosted, LoRa, 125kHz
sx1262.SetIdleCurrent(0.6)          # RC mode/XOSC off
sx1262.SetSleepCurrent(0.0012)      # Sleep with warm start/RC64k
sx1262.SetVoltage(3.3)

generateImages = 0

In [ ]:
sf = 10
bw = 125
cr = 1
dataLength = 10
PreambleLength = 500
simDurationH = 24*365*10
simDuration = simDurationH * 60*60
print("Long Preamble LoRa - relay uplink simulation - Device Consumption x Application Uplink Periodicity")
print("Simulation Duration [y]:", simDurationH/(24*365))

periodicityS = 24*60*60
periodicityS_array = np.linspace(3600, periodicityS, periodicityS-1)
periodicityMin_array = []
periodicityH_array = []

txEpCurr_array = []
rxEpCurr_array = []
sleepEpCurr_array = []
totalEpMeanCon_array = []

txRlN1Curr_array = []
rxRlN1Curr_array = []
sleepRlN1Curr_array = []
totalRlN1MeanCon_array = []

txRlN8Curr_array = []
rxRlN8Curr_array = []
sleepRlN8Curr_array = []
totalRlN8MeanCon_array = []

txRlN16Curr_array = []
rxRlN16Curr_array = []
sleepRlN16Curr_array = []
totalRlN16MeanCon_array = []

endpoint = LoRaLong(dataLength, sf, bw, cr)
relay = LoRaLong(dataLength, sf, bw, cr)

for periodicityS in periodicityS_array:

    periodicityMin_array.append(periodicityS/60)
    periodicityH_array.append(periodicityS/3600)

    txTime, rxTime, idleTime, sleepTime = endpoint.SimulateEndpointLoRaLongPreamble(periodicityS, dataLength, simDuration, PreambleLength)   
    totalTime = txTime + rxTime + idleTime + sleepTime
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    txEpCurr_array.append((tx_mAh/total_mAh)*100)
    rxEpCurr_array.append((rx_mAh/total_mAh)*100)
    sleepEpCurr_array.append((sleep_mAh/total_mAh)*100)
    totalEpMeanCon_array.append((total_mAh/(totalTime/3600)))

    n = 1
    txTime, rxTime, idleTime, sleepTime = relay.SimulateRelayLoRaLongPreamble(periodicityS, dataLength, simDuration, n, PreambleLength)
    totalTime = txTime + rxTime + idleTime + sleepTime
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    txRlN1Curr_array.append((tx_mAh/total_mAh)*100)
    rxRlN1Curr_array.append((rx_mAh/total_mAh)*100)
    sleepRlN1Curr_array.append((sleep_mAh/total_mAh)*100)
    totalRlN1MeanCon_array.append((total_mAh/(totalTime/3600)))

    n = 8
    txTime, rxTime, idleTime, sleepTime = relay.SimulateRelayLoRaLongPreamble(periodicityS, dataLength, simDuration, n, PreambleLength)   
    totalTime = txTime + rxTime + idleTime + sleepTime
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    txRlN8Curr_array.append((tx_mAh/total_mAh)*100)
    rxRlN8Curr_array.append((rx_mAh/total_mAh)*100)
    sleepRlN8Curr_array.append((sleep_mAh/total_mAh)*100)
    totalRlN8MeanCon_array.append((total_mAh/(totalTime/3600)))

    n = 16
    txTime, rxTime, idleTime, sleepTime = relay.SimulateRelayLoRaLongPreamble(periodicityS, dataLength, simDuration, n, PreambleLength)   
    totalTime = txTime + rxTime + idleTime + sleepTime
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    txRlN16Curr_array.append((tx_mAh/total_mAh)*100)
    rxRlN16Curr_array.append((rx_mAh/total_mAh)*100)
    sleepRlN16Curr_array.append((sleep_mAh/total_mAh)*100)
    totalRlN16MeanCon_array.append((total_mAh/(totalTime/3600)))


eixo_x = periodicityH_array
eixo_y1 = totalEpMeanCon_array
eixo_y2 = totalRlN1MeanCon_array
eixo_y4 = totalRlN8MeanCon_array
eixo_y5 = totalRlN16MeanCon_array

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(eixo_x, eixo_y1, color='black', linewidth=2, \
        label=f'Endpoint SF{sf} BW{bw}kHz CR{cr} Payload {dataLength} bytes')
ax.plot(eixo_x, eixo_y2, color='red', linewidth=2, \
        label=f'Relay SF{sf} BW{bw}kHz CR{cr} Payload {dataLength} bytes N = 1')
ax.plot(eixo_x, eixo_y4, color='blue', linewidth=2, \
        label=f'Relay SF{sf} BW{bw}kHz CR{cr} Payload {dataLength} bytes N = 8')
ax.plot(eixo_x, eixo_y5, color='purple', linewidth=2, \
        label=f'Relay SF{sf} BW{bw}kHz CR{cr} Payload {dataLength} bytes N = 16')
ax.set_xlabel('Uplink Periodicity (Hours) \n(b)', fontsize=16)
ax.set_ylabel('Mean Current (mA)', fontsize=16)
ax.set_yscale('log')
ax.set_ylim(0.001, 10)
ax.set_xlim(-1, 24)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.3f'))
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '' if x == -1 else f'{int(x)}'))

x_marcar = [1, 6, 12, 18]
for x_m in x_marcar:
    ax.axvline(x=0, color='black', linestyle='-', linewidth=1)
    y1_m = np.interp(x_m, eixo_x, eixo_y1)  # estima o y naquele x
    y2_m = np.interp(x_m, eixo_x, eixo_y2)  # estima o y naquele x
    y5_m = np.interp(x_m, eixo_x, eixo_y5)  # estima o y naquele x
    ax.plot(x_m, y1_m, marker='.', color='black', markersize=6)
    ax.plot(x_m, y2_m, marker='.', color='red', markersize=6)
    ax.plot(x_m, y5_m, marker='.', color='purple', markersize=6)
    ax.text(x_m, y1_m, f'{y1_m*1000:.1f}uA', fontsize=16, ha='left', va='bottom', color='black')
    ax.text(x_m, y2_m, f'{y2_m:.1f}mA', fontsize=16, ha='left', va='bottom', color='red')
    ax.text(x_m, y5_m, f'{y5_m:.1f}mA', fontsize=16, ha='left', va='bottom', color='purple')

ax.grid(True, which='both', linestyle=':', linewidth=1.5)
plt.legend()
plt.tight_layout()
# generateImages = 1
if generateImages == 1:
    plt.savefig('loralong_rl_meanCurXappPer.png', dpi=300, bbox_inches='tight', transparent=False, facecolor='white')
plt.show()

In [ ]:
sf = 10
bw = 125
cr = 1
dataLength = 10
PreambleLength = 1000
simDurationH = 24*365*10
simDuration = simDurationH * 60*60
print("Long Preamble LoRa - relay uplink simulation - Device Consumption x Application Uplink Periodicity")
print("Simulation Duration [y]:", simDurationH/(24*365))

periodicityS = 24*60*60
periodicityS_array = np.linspace(3600, periodicityS, periodicityS-1)
periodicityMin_array = []
periodicityH_array = []

txEpCurr_array = []
rxEpCurr_array = []
sleepEpCurr_array = []
totalEpMeanCon_array = []

txRlN1Curr_array = []
rxRlN1Curr_array = []
sleepRlN1Curr_array = []
totalRlN1MeanCon_array = []

txRlN8Curr_array = []
rxRlN8Curr_array = []
sleepRlN8Curr_array = []
totalRlN8MeanCon_array = []

txRlN16Curr_array = []
rxRlN16Curr_array = []
sleepRlN16Curr_array = []
totalRlN16MeanCon_array = []

endpoint = LoRaLong(dataLength, sf, bw, cr)
relay = LoRaLong(dataLength, sf, bw, cr)

for periodicityS in periodicityS_array:

    periodicityMin_array.append(periodicityS/60)
    periodicityH_array.append(periodicityS/3600)

    txTime, rxTime, idleTime, sleepTime = endpoint.SimulateEndpointLoRaLongPreamble(periodicityS, dataLength, simDuration, PreambleLength)   
    totalTime = txTime + rxTime + idleTime + sleepTime
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    txEpCurr_array.append((tx_mAh/total_mAh)*100)
    rxEpCurr_array.append((rx_mAh/total_mAh)*100)
    sleepEpCurr_array.append((sleep_mAh/total_mAh)*100)
    totalEpMeanCon_array.append((total_mAh/(totalTime/3600)))

    n = 1
    txTime, rxTime, idleTime, sleepTime = relay.SimulateRelayLoRaLongPreamble(periodicityS, dataLength, simDuration, n, PreambleLength)
    totalTime = txTime + rxTime + idleTime + sleepTime
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    txRlN1Curr_array.append((tx_mAh/total_mAh)*100)
    rxRlN1Curr_array.append((rx_mAh/total_mAh)*100)
    sleepRlN1Curr_array.append((sleep_mAh/total_mAh)*100)
    totalRlN1MeanCon_array.append((total_mAh/(totalTime/3600)))

    n = 8
    txTime, rxTime, idleTime, sleepTime = relay.SimulateRelayLoRaLongPreamble(periodicityS, dataLength, simDuration, n, PreambleLength)   
    totalTime = txTime + rxTime + idleTime + sleepTime
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    txRlN8Curr_array.append((tx_mAh/total_mAh)*100)
    rxRlN8Curr_array.append((rx_mAh/total_mAh)*100)
    sleepRlN8Curr_array.append((sleep_mAh/total_mAh)*100)
    totalRlN8MeanCon_array.append((total_mAh/(totalTime/3600)))

    n = 16
    txTime, rxTime, idleTime, sleepTime = relay.SimulateRelayLoRaLongPreamble(periodicityS, dataLength, simDuration, n, PreambleLength)   
    totalTime = txTime + rxTime + idleTime + sleepTime
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    txRlN16Curr_array.append((tx_mAh/total_mAh)*100)
    rxRlN16Curr_array.append((rx_mAh/total_mAh)*100)
    sleepRlN16Curr_array.append((sleep_mAh/total_mAh)*100)
    totalRlN16MeanCon_array.append((total_mAh/(totalTime/3600)))


eixo_x = periodicityH_array
eixo_y1 = totalEpMeanCon_array
eixo_y2 = totalRlN1MeanCon_array
eixo_y4 = totalRlN8MeanCon_array
eixo_y5 = totalRlN16MeanCon_array

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(eixo_x, eixo_y1, color='black', linewidth=2, \
        label=f'Endpoint SF{sf} BW{bw}kHz CR{cr} Payload {dataLength} bytes')
ax.plot(eixo_x, eixo_y2, color='red', linewidth=2, \
        label=f'Relay SF{sf} BW{bw}kHz CR{cr} Payload {dataLength} bytes N = 1')
ax.plot(eixo_x, eixo_y4, color='blue', linewidth=2, \
        label=f'Relay SF{sf} BW{bw}kHz CR{cr} Payload {dataLength} bytes N = 8')
ax.plot(eixo_x, eixo_y5, color='purple', linewidth=2, \
        label=f'Relay SF{sf} BW{bw}kHz CR{cr} Payload {dataLength} bytes N = 16')
ax.set_xlabel('Uplink Periodicity (Hours) \n(b)', fontsize=16)
ax.set_ylabel('Mean Current (mA)', fontsize=16)
ax.set_yscale('log')
ax.set_ylim(0.001, 10)
ax.set_xlim(-1, 24)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.3f'))
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '' if x == -1 else f'{int(x)}'))

x_marcar = [1, 6, 12, 18]
for x_m in x_marcar:
    ax.axvline(x=0, color='black', linestyle='-', linewidth=1)
    y1_m = np.interp(x_m, eixo_x, eixo_y1)  # estima o y naquele x
    y2_m = np.interp(x_m, eixo_x, eixo_y2)  # estima o y naquele x
    y5_m = np.interp(x_m, eixo_x, eixo_y5)  # estima o y naquele x
    ax.plot(x_m, y1_m, marker='.', color='black', markersize=6)
    ax.plot(x_m, y2_m, marker='.', color='red', markersize=6)
    ax.plot(x_m, y5_m, marker='.', color='purple', markersize=6)
    ax.text(x_m, y1_m, f'{y1_m*1000:.1f}uA', fontsize=16, ha='left', va='bottom', color='black')
    ax.text(x_m, y2_m, f'{y2_m:.1f}mA', fontsize=16, ha='left', va='bottom', color='red')
    ax.text(x_m, y5_m, f'{y5_m:.1f}mA', fontsize=16, ha='left', va='bottom', color='purple')

ax.grid(True, which='both', linestyle=':', linewidth=1.5)
plt.legend()
plt.tight_layout()
# generateImages = 1
if generateImages == 1:
    plt.savefig('loralong_rl_meanCurXappPer.png', dpi=300, bbox_inches='tight', transparent=False, facecolor='white')
plt.show()

In [ ]:
sf = 10
bw = 125
cr = 1
dataLength = 10
PreambleLength = 2000
simDurationH = 24*365*10
simDuration = simDurationH * 60*60
print("Long Preamble LoRa - relay uplink simulation - Device Consumption x Application Uplink Periodicity")
print("Simulation Duration [y]:", simDurationH/(24*365))

periodicityS = 24*60*60
periodicityS_array = np.linspace(3600, periodicityS, periodicityS-1)
periodicityMin_array = []
periodicityH_array = []

txEpCurr_array = []
rxEpCurr_array = []
sleepEpCurr_array = []
totalEpMeanCon_array = []

txRlN1Curr_array = []
rxRlN1Curr_array = []
sleepRlN1Curr_array = []
totalRlN1MeanCon_array = []

txRlN8Curr_array = []
rxRlN8Curr_array = []
sleepRlN8Curr_array = []
totalRlN8MeanCon_array = []

txRlN16Curr_array = []
rxRlN16Curr_array = []
sleepRlN16Curr_array = []
totalRlN16MeanCon_array = []

endpoint = LoRaLong(dataLength, sf, bw, cr)
relay = LoRaLong(dataLength, sf, bw, cr)

for periodicityS in periodicityS_array:

    periodicityMin_array.append(periodicityS/60)
    periodicityH_array.append(periodicityS/3600)

    txTime, rxTime, idleTime, sleepTime = endpoint.SimulateEndpointLoRaLongPreamble(periodicityS, dataLength, simDuration, PreambleLength)   
    totalTime = txTime + rxTime + idleTime + sleepTime
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    txEpCurr_array.append((tx_mAh/total_mAh)*100)
    rxEpCurr_array.append((rx_mAh/total_mAh)*100)
    sleepEpCurr_array.append((sleep_mAh/total_mAh)*100)
    totalEpMeanCon_array.append((total_mAh/(totalTime/3600)))

    n = 1
    txTime, rxTime, idleTime, sleepTime = relay.SimulateRelayLoRaLongPreamble(periodicityS, dataLength, simDuration, n, PreambleLength)
    totalTime = txTime + rxTime + idleTime + sleepTime
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    txRlN1Curr_array.append((tx_mAh/total_mAh)*100)
    rxRlN1Curr_array.append((rx_mAh/total_mAh)*100)
    sleepRlN1Curr_array.append((sleep_mAh/total_mAh)*100)
    totalRlN1MeanCon_array.append((total_mAh/(totalTime/3600)))

    n = 8
    txTime, rxTime, idleTime, sleepTime = relay.SimulateRelayLoRaLongPreamble(periodicityS, dataLength, simDuration, n, PreambleLength)   
    totalTime = txTime + rxTime + idleTime + sleepTime
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    txRlN8Curr_array.append((tx_mAh/total_mAh)*100)
    rxRlN8Curr_array.append((rx_mAh/total_mAh)*100)
    sleepRlN8Curr_array.append((sleep_mAh/total_mAh)*100)
    totalRlN8MeanCon_array.append((total_mAh/(totalTime/3600)))

    n = 16
    txTime, rxTime, idleTime, sleepTime = relay.SimulateRelayLoRaLongPreamble(periodicityS, dataLength, simDuration, n, PreambleLength)   
    totalTime = txTime + rxTime + idleTime + sleepTime
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    txRlN16Curr_array.append((tx_mAh/total_mAh)*100)
    rxRlN16Curr_array.append((rx_mAh/total_mAh)*100)
    sleepRlN16Curr_array.append((sleep_mAh/total_mAh)*100)
    totalRlN16MeanCon_array.append((total_mAh/(totalTime/3600)))


eixo_x = periodicityH_array
eixo_y1 = totalEpMeanCon_array
eixo_y2 = totalRlN1MeanCon_array
eixo_y4 = totalRlN8MeanCon_array
eixo_y5 = totalRlN16MeanCon_array

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(eixo_x, eixo_y1, color='black', linewidth=2, \
        label=f'Endpoint SF{sf} BW{bw}kHz CR{cr} Payload {dataLength} bytes')
ax.plot(eixo_x, eixo_y2, color='red', linewidth=2, \
        label=f'Relay SF{sf} BW{bw}kHz CR{cr} Payload {dataLength} bytes N = 1')
ax.plot(eixo_x, eixo_y4, color='blue', linewidth=2, \
        label=f'Relay SF{sf} BW{bw}kHz CR{cr} Payload {dataLength} bytes N = 8')
ax.plot(eixo_x, eixo_y5, color='purple', linewidth=2, \
        label=f'Relay SF{sf} BW{bw}kHz CR{cr} Payload {dataLength} bytes N = 16')
ax.set_xlabel('Uplink Periodicity (Hours) \n(b)', fontsize=16)
ax.set_ylabel('Mean Current (mA)', fontsize=16)
ax.set_yscale('log')
ax.set_ylim(0.001, 10)
ax.set_xlim(-1, 24)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.3f'))
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '' if x == -1 else f'{int(x)}'))

x_marcar = [1, 6, 12, 18]
for x_m in x_marcar:
    ax.axvline(x=0, color='black', linestyle='-', linewidth=1)
    y1_m = np.interp(x_m, eixo_x, eixo_y1)  # estima o y naquele x
    y2_m = np.interp(x_m, eixo_x, eixo_y2)  # estima o y naquele x
    y5_m = np.interp(x_m, eixo_x, eixo_y5)  # estima o y naquele x
    ax.plot(x_m, y1_m, marker='.', color='black', markersize=6)
    ax.plot(x_m, y2_m, marker='.', color='red', markersize=6)
    ax.plot(x_m, y5_m, marker='.', color='purple', markersize=6)
    ax.text(x_m, y1_m, f'{y1_m*1000:.1f}uA', fontsize=16, ha='left', va='bottom', color='black')
    ax.text(x_m, y2_m, f'{y2_m:.1f}mA', fontsize=16, ha='left', va='bottom', color='red')
    ax.text(x_m, y5_m, f'{y5_m:.1f}mA', fontsize=16, ha='left', va='bottom', color='purple')

ax.grid(True, which='both', linestyle=':', linewidth=1.5)
plt.legend()
plt.tight_layout()
# generateImages = 1
if generateImages == 1:
    plt.savefig('loralong_rl_meanCurXappPer.png', dpi=300, bbox_inches='tight', transparent=False, facecolor='white')
plt.show()